https://docs.sqlalchemy.org/en/13/orm/tutorial.html

In [30]:
import sqlite3

In [31]:
conn = sqlite3.connect('data/sqlite3/test.db')
conn.execute('DROP TABLE IF EXISTS User')
conn.commit()

In [32]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship

In [33]:
from sqlalchemy import create_engine
eng = create_engine('sqlite:////Users/julian/Documents/repos/SQL/data/sqlite3/test.db')

# Defino modelo

In [34]:
Base = declarative_base()

class User(Base):
    __tablename__ = 'user'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)
    age = Column(Integer)
    
    def __repr__(self):
        return f"<User(name='{self.name}', fullname='{self.fullname}', nickname='{self.nickname}'), age='{self.age}'>"

# Creo la tabla

In [35]:
# nos crea automaticamente el Schema
User.__table__ 

Table('user', MetaData(), Column('id', Integer(), table=<user>, primary_key=True, nullable=False), Column('name', String(), table=<user>), Column('fullname', String(), table=<user>), Column('nickname', String(), table=<user>), Column('age', Integer(), table=<user>), schema=None)

In [36]:
Base.metadata.create_all(eng)

In [37]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname', age=25)

In [38]:
ed_user.name, ed_user.fullname, ed_user.nickname, ed_user.age, ed_user.id

('ed', 'Ed Jones', 'edsnickname', 25, None)

# Creación de la sessión

In [39]:
from sqlalchemy.orm import sessionmaker
session = sessionmaker(bind=eng)()

# Agregamos usuario

In [40]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname', age=25)
session.add(ed_user)

# Query

In [41]:
query = session.query(User).filter_by(name='ed')

In [42]:
# all, count, first
query.first()

<User(name='ed', fullname='Ed Jones', nickname='edsnickname'), age='25'>

In [43]:
session.query(User).count()

1

# Agregamos mas usuarios

In [44]:
# Agregamos otros usuarios
session.add_all([User(name='wendy', fullname='Wendy Williams', nickname='windy', age=20),
                 User(name='mary', fullname='Mary Contrary', nickname='mary', age=50),
                 User(name='john', fullname='John Lennon', nickname='John', age=50),
                 User(name='fred', fullname='Fred Flintstone', nickname='freddy', age=80)])

In [45]:
# session.query(User).count()

In [46]:
# Le modificamos el nombre a ed
ed_user.nickname = 'eddie'

In [47]:
session.dirty

IdentitySet([<User(name='ed', fullname='Ed Jones', nickname='eddie'), age='25'>])

In [48]:
session.new

IdentitySet([<User(name='wendy', fullname='Wendy Williams', nickname='windy'), age='20'>, <User(name='mary', fullname='Mary Contrary', nickname='mary'), age='50'>, <User(name='john', fullname='John Lennon', nickname='John'), age='50'>, <User(name='fred', fullname='Fred Flintstone', nickname='freddy'), age='80'>])

In [49]:
session.commit()

In [50]:
session.dirty

IdentitySet([])

In [51]:
for q in session.query(User):
    print(q)

<User(name='ed', fullname='Ed Jones', nickname='eddie'), age='25'>
<User(name='wendy', fullname='Wendy Williams', nickname='windy'), age='20'>
<User(name='mary', fullname='Mary Contrary', nickname='mary'), age='50'>
<User(name='john', fullname='John Lennon', nickname='John'), age='50'>
<User(name='fred', fullname='Fred Flintstone', nickname='freddy'), age='80'>


# Volviendo atras (Rollback)

In [53]:
ed_user.name = 'Edwardo'

In [54]:
fake_user = User(name='fakeuser', fullname='Invalid', nickname='12345')
session.add(fake_user)

In [55]:
session.dirty

IdentitySet([<User(name='Edwardo', fullname='Ed Jones', nickname='eddie'), age='25'>])

In [56]:
session.new

IdentitySet([<User(name='fakeuser', fullname='Invalid', nickname='12345'), age='None'>])

In [57]:
session.rollback()

In [58]:
session.dirty

IdentitySet([])

In [59]:
session.new

IdentitySet([])

# Ordenando

In [60]:
session.query(User).order_by(User.age).all()

[<User(name='wendy', fullname='Wendy Williams', nickname='windy'), age='20'>,
 <User(name='ed', fullname='Ed Jones', nickname='eddie'), age='25'>,
 <User(name='mary', fullname='Mary Contrary', nickname='mary'), age='50'>,
 <User(name='john', fullname='John Lennon', nickname='John'), age='50'>,
 <User(name='fred', fullname='Fred Flintstone', nickname='freddy'), age='80'>]

In [61]:
for instance in session.query(User).order_by(User.age):
    print(instance.name, instance.fullname, instance.age)

wendy Wendy Williams 20
ed Ed Jones 25
mary Mary Contrary 50
john John Lennon 50
fred Fred Flintstone 80


# Agregación

In [62]:
import sqlalchemy

session.query(User.age, sqlalchemy.func.count(User.age)).group_by(User.age).all()

[(20, 1), (25, 1), (50, 2), (80, 1)]

In [38]:
session.query(sqlalchemy.func.avg(User.age)).all()

[(45.0,)]

# Filtros

In [39]:
session.query(User).filter(User.age == 50).all()

[<User(name='mary', fullname='Mary Contrary', nickname='mary'), age='50'>,
 <User(name='john', fullname='John Lennon', nickname='John'), age='50'>]